In [40]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.functions import to_date, year, month, dayofmonth, dayofweek, hour
from pyspark.sql.functions import countDistinct, avg, col
from pyspark.sql import Window

In [2]:
spark = SparkSession.builder \
    .appName("DivvyBikes_EDA") \
    .master("local[*]") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.maxResultSize", "2g") \
    .config("spark.sql.shuffle.partitions", "100") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .getOrCreate()

In [3]:
spark

In [9]:
# Определяем схему явно
schema = StructType([
    StructField("ride_id", StringType(), True),
    StructField("rideable_type", StringType(), True),
    StructField("started_at", TimestampType(), True),
    StructField("ended_at", TimestampType(), True),
    StructField("start_station_name", StringType(), True),
    StructField("start_station_id", StringType(), True),
    StructField("end_station_name", StringType(), True),
    StructField("end_station_id", StringType(), True),
    StructField("start_lat", DoubleType(), True),
    StructField("start_lng", DoubleType(), True),
    StructField("end_lat", DoubleType(), True),
    StructField("end_lng", DoubleType(), True),
    StructField("member_casual", StringType(), True)
])

df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .option("timestampFormat", "yyyy-MM-dd HH:mm:ss") \
    .option("encoding", "UTF-8") \
    .option("mode", "DROPMALFORMED") \
    .csv("output.csv")

# Сохраняем в Parquet для быстрого доступа в будущем
df.write \
    .mode("overwrite") \
    .option("compression", "snappy") \
    .parquet("divvy_data.parquet")

In [14]:
print("\n1. ОСНОВНАЯ ИНФОРМАЦИЯ:")
print()

# Количество строк и столбцов
row_count = df.count()
col_count = len(df.columns)
print(f"Всего записей: {row_count:,}")
print(f"Количество столбцов: {col_count}")

print(f"Количество партиций: {df.rdd.getNumPartitions()}")

# 2. СТРУКТУРА ДАННЫХ
print("\n2. СТРУКТУРА ДАННЫХ:")
df.printSchema()

# 3. ПРОСМОТР ДАННЫХ
print("\n3. ПЕРВЫЕ 10 ЗАПИСЕЙ:")
print("-" * 40)
df.show(10, truncate=True)

# 4. ПРОВЕРКА ПРОПУЩЕННЫХ ЗНАЧЕНИЙ
print("\n4. АНАЛИЗ ПРОПУЩЕННЫХ ЗНАЧЕНИЙ:")
print()

# Используем только isnull() для всех типов данных
null_counts = df.select([
    count(when(col(c).isNull(), c)).alias(c) 
    for c in df.columns
])

null_data = null_counts.collect()[0].asDict()

print(f"{'Столбец':<25} {'NULL значения':<15} {'% от общего':<15}")
print("-" * 55)

for column, null_count in null_data.items():
    if null_count > 0:
        percentage = (null_count / row_count) * 100
        print(f"{column:<25} {null_count:<15,} {percentage:.2f}%")
    else:
        print(f"{column:<25} {'0':<15} {'0.00%':<15}")

# 5. БАЗОВАЯ СТАТИСТИКА ПО ЧИСЛОВЫМ ПОЛЯМ
print("\n5. СТАТИСТИКА ПО ЧИСЛОВЫМ ПОЛЯМ:")
print()

# Определяем числовые столбцы
numeric_cols = [f.name for f in df.schema.fields 
                if isinstance(f.dataType, (IntegerType, DoubleType, FloatType, LongType))]

if numeric_cols:
    print("Числовые столбцы:", numeric_cols)
    df.select(numeric_cols).describe().show()
else:
    print("Числовые столбцы не найдены")

# 6. АНАЛИЗ КАТЕГОРИАЛЬНЫХ ПЕРЕМЕННЫХ
print("\n6. АНАЛИЗ КАТЕГОРИАЛЬНЫХ ПЕРЕМЕННЫХ:")
print()

categorical_cols = ["rideable_type", "member_casual"]

for cat_col in categorical_cols:
    print(f"\nРаспределение по столбцу '{cat_col}':")
    
    # Количество уникальных значений
    unique_count = df.select(cat_col).distinct().count()
    print(f"  Уникальных значений: {unique_count}")
    
    # Распределение значений
    dist_df = df.groupBy(cat_col).count().orderBy(col("count").desc())
    
    dist_df.show(truncate=False)
    
    print("  Процентное распределение:")
    dist_df.withColumn("percentage", 
                       round(col("count") / row_count * 100,
                             2)).show(truncate=False)

# 7. АНАЛИЗ ВРЕМЕННЫХ ПОЛЕЙ
print("\n7. АНАЛИЗ ВРЕМЕННЫХ ДАННЫХ:")
print()

# Добавляем вычисляемые поля для анализа

df_with_features = df.withColumn("ride_date", to_date(col("started_at"))) \
    .withColumn("ride_year", year(col("started_at"))) \
    .withColumn("ride_month", month(col("started_at"))) \
    .withColumn("ride_day", dayofmonth(col("started_at"))) \
    .withColumn("ride_dayofweek", dayofweek(col("started_at"))) \
    .withColumn("ride_hour", hour(col("started_at"))) \
    .withColumn("ride_duration_seconds", 
               (col("ended_at").cast("long") - col("started_at").cast("long"))) \
    .withColumn("ride_duration_minutes", 
               (col("ended_at").cast("long") - col("started_at").cast("long")) / 60.0)

# Временной диапазон
print("Временной диапазон поездок:")
time_range = df_with_features.agg(min("started_at").alias("first_ride"),
                                  max("started_at").alias("last_ride")).collect()[0]

print(f"Первая поездка: {time_range['first_ride']}")
print(f"Последняя поездка: {time_range['last_ride']}")

# Распределение по годам
print("\nКоличество поездок по годам:")
yearly_counts = df_with_features.groupBy("ride_year").count().orderBy("ride_year")
yearly_counts.show()

# Распределение по месяцам (все года)
print("\nКоличество поездок по месяцам:")
monthly_counts = df_with_features.groupBy("ride_month").count().orderBy("ride_month")
monthly_counts.show()

# Распределение по дням недели
print("\nКоличество поездок по дням недели (1=Вс, 7=Сб):")
dow_counts = df_with_features.groupBy("ride_dayofweek").count().orderBy("ride_dayofweek")
dow_counts.show()


1. ОСНОВНАЯ ИНФОРМАЦИЯ:

Всего записей: 31,013,853
Количество столбцов: 13
Количество партиций: 45

2. СТРУКТУРА ДАННЫХ:
root
 |-- ride_id: string (nullable = true)
 |-- rideable_type: string (nullable = true)
 |-- started_at: timestamp (nullable = true)
 |-- ended_at: timestamp (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_id: string (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_id: string (nullable = true)
 |-- start_lat: double (nullable = true)
 |-- start_lng: double (nullable = true)
 |-- end_lat: double (nullable = true)
 |-- end_lng: double (nullable = true)
 |-- member_casual: string (nullable = true)


3. ПЕРВЫЕ 10 ЗАПИСЕЙ:
----------------------------------------
+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------+---------+---------+-------+--------+-------------+
|         ride_id|rideable_type|         

In [15]:
# Распределение по часам
print("\nКоличество поездок по часам суток:")
hourly_counts = df_with_features.groupBy("ride_hour").count().orderBy("ride_hour")
hourly_counts.show(24)


Количество поездок по часам суток:
+---------+-------+
|ride_hour|  count|
+---------+-------+
|        0| 289859|
|        1| 187758|
|        2| 112867|
|        3|  66021|
|        4|  59575|
|        5| 165818|
|        6| 464877|
|        7| 848729|
|        8|1046043|
|        9| 846161|
|       10| 904161|
|       11|1123707|
|       12|1315744|
|       13|1342530|
|       14|1369821|
|       15|1559712|
|       16|1900445|
|       17|2212138|
|       18|1869609|
|       19|1366174|
|       20| 955203|
|       21| 744268|
|       22| 609325|
|       23| 431150|
+---------+-------+



In [23]:
# 8. АНАЛИЗ ДЛИТЕЛЬНОСТИ ПОЕЗДОК
print("\n8. АНАЛИЗ ДЛИТЕЛЬНОСТИ ПОЕЗДОК:")
print()

# Статистика по длительности
duration_stats = df_with_features.select(count("*").alias("total_rides"),
                                         mean("ride_duration_minutes").alias("avg_duration_min"),
                                         stddev("ride_duration_minutes").alias("std_duration_min"),
                                         min("ride_duration_minutes").alias("min_duration_min"),
                                         max("ride_duration_minutes").alias("max_duration_min"),
                                         expr(
                                             "percentile_approx(ride_duration_minutes, 0.5)").alias(
                                             "median_duration_min"),
                                         expr(
                                             "percentile_approx(ride_duration_minutes, 0.25)").alias(
                                             "q1_duration_min"),
                                         expr(
                                             "percentile_approx(ride_duration_minutes, 0.75)").alias(
                                             "q3_duration_min")).collect()[0]

print("Общая статистика:")
for stat, value in duration_stats.asDict().items():
    if "duration" in stat:
        print(f"  {stat}: {value:.2f} минут" if isinstance(value, float) else f"  {stat}: {value}")

# Проверка аномалий в длительности
print("\nПроверка аномальных значений:")
negative_duration = df_with_features.filter(col("ride_duration_seconds") < 0).count()
zero_duration = df_with_features.filter(col("ride_duration_seconds") == 0).count()
very_short = df_with_features.filter((col("ride_duration_seconds") > 0) &
                                     (col("ride_duration_seconds") < 60)).count()
very_long = df_with_features.filter(col("ride_duration_minutes") > 1440).count()

print(f"Поездок с отрицательной длительностью: {negative_duration:,}")
print(f"Поездок с нулевой длительностью: {zero_duration:,}")
print(f"Поездок короче 1 минуты: {very_short:,}")
print(f"Поездок длиннее 24 часов: {very_long:,}")



8. АНАЛИЗ ДЛИТЕЛЬНОСТИ ПОЕЗДОК:

Общая статистика:
  avg_duration_min: 20.38 минут
  std_duration_min: 221.72 минут
  min_duration_min: -29049.97 минут
  max_duration_min: 98489.07 минут
  median_duration_min: 10.92 минут
  q1_duration_min: 6.10 минут
  q3_duration_min: 19.95 минут

Проверка аномальных значений:
Поездок с отрицательной длительностью: 11,251
Поездок с нулевой длительностью: 2,798
Поездок короче 1 минуты: 437,513
Поездок длиннее 24 часов: 20,814


In [ ]:
# 9. АНАЛИЗ СТАНЦИЙ

In [28]:
# 9.1. Топ станций отправления
print("\n1. ТОП-10 СТАНЦИЙ ОТПРАВЛЕНИЯ:")
print()

top_start_stations = df_with_features.filter(col("start_station_name").isNotNull()) \
    .groupBy("start_station_name") \
    .agg(
        count("*").alias("departure_count"),
        avg("ride_duration_minutes").alias("avg_duration_min"),
        expr("percentile_approx(ride_duration_minutes, 0.5)").alias("median_duration_min")
    ) \
    .orderBy(col("departure_count").desc()) \
    .limit(10)

print("Самые популярные станции для начала поездок:")
top_start_stations.show(truncate=False)
print()


1. ТОП-10 СТАНЦИЙ ОТПРАВЛЕНИЯ:

Самые популярные станции для начала поездок:
+------------------------+---------------+------------------+-------------------+
|start_station_name      |departure_count|avg_duration_min  |median_duration_min|
+------------------------+---------------+------------------+-------------------+
|Streeter Dr & Grand Ave |270641         |42.01799191548952 |24.316666666666666 |
|Clark St & Elm St       |151471         |17.387819010459644|10.3               |
|Michigan Ave & Oak St   |150769         |35.541243889659   |22.1               |
|Wells St & Concord Ln   |146309         |16.069703390313208|10.5               |
|Millennium Park         |137675         |50.77255045094124 |24.0               |
|Theater on the Lake     |135042         |29.3553411778064  |20.166666666666668 |
|Kingsbury St & Kinzie St|131698         |12.112822391633392|7.666666666666667  |
|Wells St & Elm St       |129395         |15.427333874312495|9.083333333333334  |
|Broadway & Barry Av

In [29]:
# 9.2. Топ станций прибытия
print("\n2. ТОП-10 СТАНЦИЙ ПРИБЫТИЯ:")
print()

top_end_stations = df_with_features.filter(col("end_station_name").isNotNull()) \
    .groupBy("end_station_name") \
    .agg(
        count("*").alias("arrival_count"),
        avg("ride_duration_minutes").alias("avg_duration_min"),
        expr("percentile_approx(ride_duration_minutes, 0.5)").alias("median_duration_min")
    ) \
    .orderBy(col("arrival_count").desc()) \
    .limit(10)

print("Самые популярные станции для завершения поездок:")
top_end_stations.show(truncate=False)
print()


2. ТОП-10 СТАНЦИЙ ПРИБЫТИЯ:

Самые популярные станции для завершения поездок:
+----------------------------------+-------------+------------------+-------------------+
|end_station_name                  |arrival_count|avg_duration_min  |median_duration_min|
+----------------------------------+-------------+------------------+-------------------+
|Streeter Dr & Grand Ave           |274429       |35.80527750347083 |26.05              |
|Michigan Ave & Oak St             |152545       |32.00033924415746 |22.8               |
|Clark St & Elm St                 |149126       |15.219492129697922|10.35              |
|Wells St & Concord Ln             |147470       |14.280419407337078|10.416666666666666 |
|Millennium Park                   |140452       |34.22014555387841 |22.9               |
|Theater on the Lake               |137120       |30.220208211785284|23.466666666666665 |
|Kingsbury St & Kinzie St          |128169       |11.297236201161487|7.566666666666666  |
|Wells St & Elm St   

In [30]:
# 9.3. Самые активные станции
print("\n3. ТОП-10 САМЫХ АКТИВНЫХ СТАНЦИЙ:")
print("-" * 40)

# Создаем DataFrame с отправлениями
departures = df_with_features.filter(col("start_station_name").isNotNull()) \
    .select(col("start_station_name").alias("station_name")) \
    .withColumn("activity_type", lit("departure"))

# Создаем DataFrame с прибытиями
arrivals = df_with_features.filter(col("end_station_name").isNotNull()) \
    .select(col("end_station_name").alias("station_name")) \
    .withColumn("activity_type", lit("arrival"))

# Объединяем и считаем
station_activity = departures.unionAll(arrivals) \
    .groupBy("station_name") \
    .agg(
        count("*").alias("total_activity"),
        sum(when(col("activity_type") == "departure", 1).otherwise(0)).alias("departures"),
        sum(when(col("activity_type") == "arrival", 1).otherwise(0)).alias("arrivals")
    ) \
    .orderBy(col("total_activity").desc()) \
    .limit(10)

print("Самые активные станции (сумма отправлений и прибытий):")
station_activity.show(truncate=False)
print()


3. ТОП-10 САМЫХ АКТИВНЫХ СТАНЦИЙ:
----------------------------------------
Самые активные станции (сумма отправлений и прибытий):
+----------------------------------+--------------+----------+--------+
|station_name                      |total_activity|departures|arrivals|
+----------------------------------+--------------+----------+--------+
|Streeter Dr & Grand Ave           |705308        |350188    |355120  |
|Michigan Ave & Oak St             |430561        |214654    |215907  |
|Clark St & Elm St                 |405535        |204435    |201100  |
|Wells St & Concord Ln             |390088        |194420    |195668  |
|Kingsbury St & Kinzie St          |389233        |196835    |192398  |
|Millennium Park                   |379923        |188530    |191393  |
|Theater on the Lake               |371533        |184038    |187495  |
|DuSable Lake Shore Dr & Monroe St |361283        |184109    |177174  |
|DuSable Lake Shore Dr & North Blvd|357665        |172568    |185097  |
|Well

In [31]:
# 9.4. Станции с наибольшим дисбалансом
print("\n4. СТАНЦИИ С НАИБОЛЬШИМ ДИСБАЛАНСОМ:")
print()

station_balance = departures.unionAll(arrivals) \
    .groupBy("station_name") \
    .agg(
        sum(when(col("activity_type") == "departure", 1).otherwise(0)).alias("departures"),
        sum(when(col("activity_type") == "arrival", 1).otherwise(0)).alias("arrivals")
    ) \
    .withColumn("net_flow", col("departures") - col("arrivals")) \
    .withColumn("status",
                when(col("net_flow") > 0, "Дефицит велосипедов")
                .when(col("net_flow") < 0, "Избыток велосипедов")
                .otherwise("Баланс")) \
    .filter(col("status") != "Баланс") \
    .orderBy(abs(col("net_flow")).desc()) \
    .limit(10)

print("Станции, куда нужно добавлять или убирать велосипеды:")
station_balance.show(truncate=False)
print()


4. СТАНЦИИ С НАИБОЛЬШИМ ДИСБАЛАНСОМ:

Станции, куда нужно добавлять или убирать велосипеды:
+----------------------------------+----------+--------+--------+-------------------+
|station_name                      |departures|arrivals|net_flow|status             |
+----------------------------------+----------+--------+--------+-------------------+
|Columbus Dr & Randolph St         |125096    |105696  |19400   |Дефицит велосипедов|
|Sheffield Ave & Waveland Ave      |104018    |117359  |-13341  |Избыток велосипедов|
|DuSable Lake Shore Dr & North Blvd|172568    |185097  |-12529  |Избыток велосипедов|
|Desplaines St & Kinzie St         |131660    |120094  |11566   |Дефицит велосипедов|
|Sedgwick St & Huron St            |79427     |69494   |9933    |Дефицит велосипедов|
|Field Museum                      |54826     |45011   |9815    |Дефицит велосипедов|
|Wells St & Walton St              |64176     |54479   |9697    |Дефицит велосипедов|
|Cityfront Plaza Dr & Pioneer Ct   |76485     |

In [32]:
# 9.5. Анализ самых популярных маршрутов
print("\n5. ТОП-10 САМЫХ ПОПУЛЯРНЫХ МАРШРУТОВ:")
print()

top_routes = df_with_features.filter(
    col("start_station_name").isNotNull() &
    col("end_station_name").isNotNull() &
    (col("start_station_name") != col("end_station_name"))
).groupBy("start_station_name", "end_station_name").agg(
    count("*").alias("route_count"),
    avg("ride_duration_minutes").alias("avg_duration_min"),
    expr("percentile_approx(ride_duration_minutes, 0.5)").alias("median_duration_min"),
    min("ride_duration_minutes").alias("min_duration_min"),
    max("ride_duration_minutes").alias("max_duration_min")
).orderBy(col("route_count").desc()).limit(10)

print("Самые популярные маршруты между станциями:")
top_routes.show(truncate=False)
print()


5. ТОП-10 САМЫХ ПОПУЛЯРНЫХ МАРШРУТОВ:

Самые популярные маршруты между станциями:
+---------------------------------+------------------------+-----------+------------------+-------------------+--------------------+------------------+
|start_station_name               |end_station_name        |route_count|avg_duration_min  |median_duration_min|min_duration_min    |max_duration_min  |
+---------------------------------+------------------------+-----------+------------------+-------------------+--------------------+------------------+
|Ellis Ave & 60th St              |Ellis Ave & 55th St     |21114      |5.754140222916864 |4.433333333333334  |2.15                |1384.9            |
|Ellis Ave & 60th St              |University Ave & 57th St|20273      |5.348130518423522 |3.65               |-0.2                |1449.0            |
|Ellis Ave & 55th St              |Ellis Ave & 60th St     |19380      |6.124352425180599 |4.633333333333334  |-6.75               |1348.8833333333334|
|Univ

In [33]:
# 9.6. Маршруты с экстремальной длительностью
print("\n6. МАРШРУТЫ С ЭКСТРЕМАЛЬНОЙ ДЛИТЕЛЬНОСТЬЮ:")
print()

# Маршруты с самой большой средней длительностью
print(" Маршруты с самой большой средней длительностью (минимум 50 поездок):")

long_routes = df_with_features.filter(
    col("start_station_name").isNotNull() &
    col("end_station_name").isNotNull() &
    (col("start_station_name") != col("end_station_name")) &
    col("ride_duration_minutes").isNotNull()
).groupBy("start_station_name", "end_station_name").agg(
    count("*").alias("route_count"),
    avg("ride_duration_minutes").alias("avg_duration_min"),
    stddev("ride_duration_minutes").alias("std_duration_min")
).filter(col("route_count") >= 50).orderBy(col("avg_duration_min").desc()).limit(5)

long_routes.show(truncate=False)

# Маршруты с самой маленькой средней длительностью
print("\n• Маршруты с самой маленькой средней длительностью (минимум 50 поездок):")

short_routes = df_with_features.filter(
    col("start_station_name").isNotNull() &
    col("end_station_name").isNotNull() &
    (col("start_station_name") != col("end_station_name")) &
    col("ride_duration_minutes").isNotNull()
).groupBy("start_station_name", "end_station_name").agg(
    count("*").alias("route_count"),
    avg("ride_duration_minutes").alias("avg_duration_min"),
    stddev("ride_duration_minutes").alias("std_duration_min")
).filter(col("route_count") >= 50).orderBy(col("avg_duration_min")).limit(5)

short_routes.show(truncate=False)
print()


6. МАРШРУТЫ С ЭКСТРЕМАЛЬНОЙ ДЛИТЕЛЬНОСТЬЮ:

 Маршруты с самой большой средней длительностью (минимум 50 поездок):
+----------------------------------+---------------------------+-----------+------------------+------------------+
|start_station_name                |end_station_name           |route_count|avg_duration_min  |std_duration_min  |
+----------------------------------+---------------------------+-----------+------------------+------------------+
|Sedgwick St & Schiller St         |LaSalle St & Jackson Blvd  |69         |821.9666666666666 |6701.729630538173 |
|Franklin St & Adams St (Temp)     |Halsted St & Wrightwood Ave|53         |777.8647798742139 |5477.888633569619 |
|LaSalle St & Washington St        |Michigan Ave & 18th St     |66         |583.7361111111112 |4594.8310354271625|
|DuSable Lake Shore Dr & North Blvd|Western Ave & Winnebago Ave|74         |566.5394144144145 |4598.248999437317 |
|State St & Randolph St            |Calumet Ave & 21st St      |63         |475.

In [34]:
# 9.7. Анализ возвратов на ту же станцию
print("\n7. АНАЛИЗ ВОЗВРАТОВ НА ТУ ЖЕ СТАНЦИЮ:")
print()

same_station_stats = df_with_features.filter(
    col("start_station_name").isNotNull() &
    col("end_station_name").isNotNull() &
    (col("start_station_name") == col("end_station_name"))
).agg(
    count("*").alias("return_trips_count"),
    (count("*") / row_count * 100).alias("return_percentage"),
    avg("ride_duration_minutes").alias("avg_return_duration"),
    expr("percentile_approx(ride_duration_minutes, 0.5)").alias("median_return_duration"),
    min("ride_duration_minutes").alias("min_return_duration"),
    max("ride_duration_minutes").alias("max_return_duration")
).collect()[0]

print(f" Количество поездок на ту же станцию: {same_station_stats['return_trips_count']:,}")
print(f" Процент от общего числа: {same_station_stats['return_percentage']:.2f}%")
print(f" Средняя длительность: {same_station_stats['avg_return_duration']:.1f} минут")
print(f" Медианная длительность: {same_station_stats['median_return_duration']:.1f} минут")
print(f" Минимальная длительность: {same_station_stats['min_return_duration']:.1f} минут")
print(f" Максимальная длительность: {same_station_stats['max_return_duration']:.1f} минут")
print()


7. АНАЛИЗ ВОЗВРАТОВ НА ТУ ЖЕ СТАНЦИЮ:

 Количество поездок на ту же станцию: 1,322,119
 Процент от общего числа: 4.26%
 Средняя длительность: 35.4 минут
 Медианная длительность: 19.4 минут
 Минимальная длительность: -29034.7 минут
 Максимальная длительность: 33248.0 минут



In [35]:
# 9.8. Станции с самыми частыми возвратами
print("\n8. СТАНЦИИ С ЧАСТЫМИ ВОЗВРАТАМИ:")
print()

frequent_return_stations = df_with_features.filter(
    col("start_station_name").isNotNull() &
    col("end_station_name").isNotNull()
).withColumn(
    "is_return", 
    when(col("start_station_name") == col("end_station_name"), 1).otherwise(0)
).groupBy("start_station_name").agg(
    count("*").alias("total_departures"),
    sum("is_return").alias("return_departures"),
    (sum("is_return") / count("*") * 100).alias("return_rate_percent"),
    avg(when(col("is_return") == 1, col("ride_duration_minutes"))).alias("avg_return_duration")
).filter(col("total_departures") >= 100).orderBy(col("return_rate_percent").desc()).limit(10)

print("Станции, с которых часто возвращаются на ту же станцию:")
frequent_return_stations.show(truncate=False)
print()


8. СТАНЦИИ С ЧАСТЫМИ ВОЗВРАТАМИ:

Станции, с которых часто возвращаются на ту же станцию:
+--------------------------------------+----------------+-----------------+-------------------+-------------------+
|start_station_name                    |total_departures|return_departures|return_rate_percent|avg_return_duration|
+--------------------------------------+----------------+-----------------+-------------------+-------------------+
|WATSON TESTING - DIVVY                |2819            |2819             |100.0              |0.08630128887312226|
|Big Marsh Park                        |932             |908              |97.42489270386267  |54.12582599118942  |
|HUBBARD ST BIKE CHECKING (LBS-WH-TEST)|391             |349              |89.25831202046037  |0.07741165234001918|
|Altgeld Gardens                       |151             |128              |84.76821192052981  |92.93802083333337  |
|State St & 123rd St                   |273             |226              |82.78388278388277  |45

In [36]:
# 9.9. Анализ по типам пользователей на станциях
print("\n9. СТАНЦИИ ДЛЯ РАЗНЫХ ТИПОВ ПОЛЬЗОВАТЕЛЕЙ:")
print()

print(" Топ-5 станций для каждого типа пользователей:")

for user_type in ["member", "casual"]:
    print(f"\n  Для пользователей типа '{user_type}':")
    
    user_stations = df_with_features.filter(col("member_casual") == user_type) \
        .filter(col("start_station_name").isNotNull()) \
        .groupBy("start_station_name") \
        .agg(
            count("*").alias("departure_count"),
            avg("ride_duration_minutes").alias("avg_duration_min"),
            (count("*") / row_count * 100).alias("percentage_of_total")
        ) \
        .orderBy(col("departure_count").desc()) \
        .limit(5)
    
    user_stations.show(truncate=False)
print()


9. СТАНЦИИ ДЛЯ РАЗНЫХ ТИПОВ ПОЛЬЗОВАТЕЛЕЙ:

 Топ-5 станций для каждого типа пользователей:

  Для пользователей типа 'member':
+----------------------------+---------------+------------------+-------------------+
|start_station_name          |departure_count|avg_duration_min  |percentage_of_total|
+----------------------------+---------------+------------------+-------------------+
|Clark St & Elm St           |97485          |12.1568566104187  |0.3143272782004867 |
|Kingsbury St & Kinzie St    |96093          |8.853021898924306 |0.3098389613183502 |
|Wells St & Concord Ln       |86092          |11.93421204447955 |0.27759208119029904|
|Wells St & Elm St           |79246          |11.353714803691453|0.2555180744553087 |
|Clinton St & Washington Blvd|77575          |11.556316897625958|0.2501301595773992 |
+----------------------------+---------------+------------------+-------------------+


  Для пользователей типа 'casual':
+---------------------------------+---------------+----------

In [ ]:
# 10. ГЕОГРАФИЧЕСКИЙ АНАЛИЗ

In [37]:
# Сначала добавим недостающие колонки в df_with_features
# Создаем расширенный df_with_features с нужными колонками
df_with_geo_features = df_with_features \
    .withColumn("lat_grid", round(col("start_lat"), 2)) \
    .withColumn("lng_grid", round(col("start_lng"), 2)) \
    .withColumn("is_weekend", 
                when((dayofweek(col("started_at")) == 1) | 
                     (dayofweek(col("started_at")) == 7), True).otherwise(False)) \
    .withColumn("lat_diff", abs(col("end_lat") - col("start_lat"))) \
    .withColumn("lng_diff", abs(col("end_lng") - col("start_lng"))) \
    .withColumn("coord_diff", col("lat_diff") + col("lng_diff"))

# Создаем временное представление
df_with_geo_features.createOrReplaceTempView("divvy_rides_geo")

In [38]:
# 10.1. Поездки с пропущенными координатами
print("\n1. Поездки с пропущенными координатами (проблема качества данных):")
print()

missing_coords = df.filter(
    (col("start_lat").isNull()) |
    (col("start_lng").isNull()) |
    (col("end_lat").isNull()) |
    (col("end_lng").isNull())
).count()

print(f" Поездок с пропущенными координатами: {missing_coords:,}")
print(f" Процент от общего числа: {(missing_coords / row_count) * 100:.2f}%")


1. Поездки с пропущенными координатами (проблема качества данных):

 Поездок с пропущенными координатами: 34,161
 Процент от общего числа: 0.11%


In [41]:
# 10.2. Анализ плотности станций
print("\n2. Анализ географии использования (практичный подход):")
print()

# Анализ плотности поездок по сетке
print("\n Топ-10 зон с наибольшей плотностью поездок:")

density_grid = df_with_geo_features.filter(
    col("lat_grid").isNotNull() & col("lng_grid").isNotNull()
).groupBy("lat_grid", "lng_grid").agg(
    count("*").alias("trip_count"),
    countDistinct("start_station_name").alias("unique_stations"),
    avg("ride_duration_minutes").alias("avg_duration")
).orderBy(col("trip_count").desc()).limit(10)

print("(округленные координаты до 2 знаков после запятой)")
density_grid.show(truncate=False)


2. Анализ географии использования (практичный подход):


 Топ-10 зон с наибольшей плотностью поездок:
(округленные координаты до 2 знаков после запятой)
+--------+--------+----------+---------------+------------------+
|lat_grid|lng_grid|trip_count|unique_stations|avg_duration      |
+--------+--------+----------+---------------+------------------+
|41.89   |-87.63  |876000    |107            |21.061712119482497|
|41.88   |-87.64  |856129    |106            |15.659684521841918|
|41.9    |-87.63  |765446    |118            |17.668267933553334|
|41.89   |-87.64  |688887    |103            |12.917727411510645|
|41.88   |-87.63  |683849    |138            |22.115606466729744|
|41.89   |-87.62  |680359    |83             |25.61550125252501 |
|41.88   |-87.62  |631665    |63             |39.948253583782545|
|41.91   |-87.63  |602769    |86             |20.682474823135674|
|41.95   |-87.65  |570197    |41             |16.860396991449154|
|41.94   |-87.65  |546866    |63             |16.31312

In [42]:
# 10.3. Поездки с аномальными координатами
print("\n3. Обнаружение аномальных координат:")
print()

# Проверяем координаты на реалистичность (для Чикаго)
chicago_lat_range = (41.5, 42.0)  # примерный диапазон для Чикаго
chicago_lng_range = (-88.0, -87.5)  # примерный диапазон

anomalous_coords = df.filter(
    (col("start_lat") < chicago_lat_range[0]) |
    (col("start_lat") > chicago_lat_range[1]) |
    (col("start_lng") < chicago_lng_range[0]) |
    (col("start_lng") > chicago_lng_range[1]) |
    (col("end_lat") < chicago_lat_range[0]) |
    (col("end_lat") > chicago_lat_range[1]) |
    (col("end_lng") < chicago_lng_range[0]) |
    (col("end_lng") > chicago_lng_range[1])
).count()

print(f" Поездок с координатами вне Чикаго: {anomalous_coords:,}")
print(f" Процент: {(anomalous_coords / row_count) * 100:.2f}%")

if anomalous_coords > 0:
    print("\n Примеры аномальных координат:")
    df.filter(
        (col("start_lat") < chicago_lat_range[0]) |
        (col("start_lat") > chicago_lat_range[1])
    ).select(
        "ride_id", "start_lat", "start_lng", "end_lat", "end_lng"
    ).limit(3).show(truncate=False)


3. Обнаружение аномальных координат:

 Поездок с координатами вне Чикаго: 648,601
 Процент: 2.09%

 Примеры аномальных координат:
+----------------+---------+---------+-------+--------+
|ride_id         |start_lat|start_lng|end_lat|end_lng |
+----------------+---------+---------+-------+--------+
|4AC1B5A237958211|42.057   |-87.6866 |42.057 |-87.6866|
|083A4A77D27965B2|42.016   |-87.675  |41.984 |-87.6603|
|0993F227BFBB2830|42.0529  |-87.6734 |42.016 |-87.6686|
+----------------+---------+---------+-------+--------+



In [43]:
# 10.4. Анализ расстояний (практичный)
print("\n4. Практический анализ 'длинных' и 'коротких' поездок:")
print()

# Категоризируем поездки
print("\n Распределение поездок по категориям удаленности:")

distance_categories = df_with_geo_features.withColumn(
    "distance_category",
    when(col("coord_diff").isNull(), "Нет координат")
    .when(col("start_station_name") == col("end_station_name"), "Та же станция")
    .when(col("coord_diff") < 0.001, "Очень близко")
    .when(col("coord_diff") < 0.01, "Близко")
    .when(col("coord_diff") < 0.05, "Среднее расстояние")
    .otherwise("Далеко")
).groupBy("distance_category").agg(
    count("*").alias("trip_count"),
    avg("ride_duration_minutes").alias("avg_duration_min"),
    (count("*") / row_count * 100).alias("percentage")
).orderBy(col("trip_count").desc())

distance_categories.show(truncate=False)


4. Практический анализ 'длинных' и 'коротких' поездок:


 Распределение поездок по категориям удаленности:
+------------------+----------+------------------+-------------------+
|distance_category |trip_count|avg_duration_min  |percentage         |
+------------------+----------+------------------+-------------------+
|Среднее расстояние|13761229  |14.792991965567394|44.371233074458694 |
|Близко            |3164121   |11.812759288704015|10.20228283148179  |
|Далеко            |3145994   |34.31518183654089 |10.143834756681152 |
|Та же станция     |1322119   |35.36759394073707 |4.262994991302758  |
|Очень близко      |374094    |8.424540489823412 |1.2062158158807292 |
|Нет координат     |24138     |1877.3875590355428|0.07782973627946196|
+------------------+----------+------------------+-------------------+



In [44]:
# 10.5. Географические паттерны по времени суток
print("\n5. География поездок в разное время суток:")
print()

# Утренние часы (пик)
print("\n Топ-5 зон утренних поездок (7-9 утра):")
morning_pattern = df_with_geo_features.filter(
    (col("ride_hour") >= 7) & (col("ride_hour") <= 9)
).filter(
    col("lat_grid").isNotNull() & col("lng_grid").isNotNull()
).groupBy("lat_grid", "lng_grid").agg(
    count("*").alias("morning_trips"),
    countDistinct("start_station_name").alias("stations"),
    avg("ride_duration_minutes").alias("avg_duration_min")
).orderBy(col("morning_trips").desc()).limit(5)

morning_pattern.show(truncate=False)

# Вечерние часы (пик)
print("\n Топ-5 зон вечерних поездок (17-19 вечера):")
evening_pattern = df_with_geo_features.filter(
    (col("ride_hour") >= 17) & (col("ride_hour") <= 19)
).filter(
    col("lat_grid").isNotNull() & col("lng_grid").isNotNull()
).groupBy("lat_grid", "lng_grid").agg(
    count("*").alias("evening_trips"),
    countDistinct("end_station_name").alias("stations"),
    avg("ride_duration_minutes").alias("avg_duration_min")
).orderBy(col("evening_trips").desc()).limit(5)

evening_pattern.show(truncate=False)


5. География поездок в разное время суток:


 Топ-5 зон утренних поездок (7-9 утра):
+--------+--------+-------------+--------+------------------+
|lat_grid|lng_grid|morning_trips|stations|avg_duration_min  |
+--------+--------+-------------+--------+------------------+
|41.88   |-87.64  |169175       |38      |10.871254519481798|
|41.9    |-87.63  |113288       |44      |12.068583021443876|
|41.89   |-87.64  |111203       |37      |9.295963538153948 |
|41.89   |-87.63  |108936       |41      |13.908299368436511|
|41.89   |-87.62  |89940        |25      |17.58351771551405 |
+--------+--------+-------------+--------+------------------+


 Топ-5 зон вечерних поездок (17-19 вечера):
+--------+--------+-------------+--------+------------------+
|lat_grid|lng_grid|evening_trips|stations|avg_duration_min  |
+--------+--------+-------------+--------+------------------+
|41.88   |-87.64  |225799       |625     |16.44821876979083 |
|41.89   |-87.63  |224749       |642     |20.633427215842264|


In [45]:
# 10.6. Анализ для перераспределения велосипедов
print("\n6. Анализ для оптимизации перераспределения велосипедов:")
print("-" * 40)

print("\n• Станции с наибольшим дисбалансом:")

# Используем DataFrame API вместо SQL
# Считаем отправления
departures_count = df_with_geo_features.filter(col("start_station_name").isNotNull()) \
    .groupBy("start_station_name") \
    .agg(count("*").alias("departures"))

# Считаем прибытия
arrivals_count = df_with_geo_features.filter(col("end_station_name").isNotNull()) \
    .groupBy("end_station_name") \
    .agg(count("*").alias("arrivals"))

# Объединяем
stations_with_deficit = departures_count.alias("d").join(
    arrivals_count.alias("a"),
    col("d.start_station_name") == col("a.end_station_name"),
    "full_outer"
).select(
    coalesce(col("d.start_station_name"), col("a.end_station_name")).alias("station_name"),
    coalesce(col("d.departures"), lit(0)).alias("departures"),
    coalesce(col("a.arrivals"), lit(0)).alias("arrivals")
).withColumn(
    "net_flow", col("departures") - col("arrivals")
).withColumn(
    "status",
    when(col("net_flow") > 0, "Дефицит велосипедов")
    .when(col("net_flow") < 0, "Избыток велосипедов")
    .otherwise("Баланс")
).filter(col("status") != "Баланс") \
 .orderBy(abs(col("net_flow")).desc()) \
 .limit(10)

stations_with_deficit.show(truncate=False)


6. Анализ для оптимизации перераспределения велосипедов:
----------------------------------------

• Станции с наибольшим дисбалансом:
+----------------------------------+----------+--------+--------+-------------------+
|station_name                      |departures|arrivals|net_flow|status             |
+----------------------------------+----------+--------+--------+-------------------+
|Columbus Dr & Randolph St         |125096    |105696  |19400   |Дефицит велосипедов|
|Sheffield Ave & Waveland Ave      |104018    |117359  |-13341  |Избыток велосипедов|
|DuSable Lake Shore Dr & North Blvd|172568    |185097  |-12529  |Избыток велосипедов|
|Desplaines St & Kinzie St         |131660    |120094  |11566   |Дефицит велосипедов|
|Sedgwick St & Huron St            |79427     |69494   |9933    |Дефицит велосипедов|
|Field Museum                      |54826     |45011   |9815    |Дефицит велосипедов|
|Wells St & Walton St              |64176     |54479   |9697    |Дефицит велосипедов|
|Cit

In [46]:
# 10.7. Анализ по дням недели для планирования
print("\n7. География поездок в будни vs выходные:")
print()

print("\n Топ-3 зон в будние дни:")
weekday_geo = df_with_geo_features.filter(
    ~col("is_weekend")  # будни
).filter(
    col("lat_grid").isNotNull() & col("lng_grid").isNotNull()
).groupBy("lat_grid", "lng_grid").agg(
    count("*").alias("weekday_trips"),
    countDistinct("start_station_name").alias("stations"),
    avg("ride_duration_minutes").alias("avg_duration_min")
).orderBy(col("weekday_trips").desc()).limit(3)

weekday_geo.show(truncate=False)

print("\n Топ-3 зон в выходные дни:")
weekend_geo = df_with_geo_features.filter(
    col("is_weekend")  # выходные
).filter(
    col("lat_grid").isNotNull() & col("lng_grid").isNotNull()
).groupBy("lat_grid", "lng_grid").agg(
    count("*").alias("weekend_trips"),
    countDistinct("start_station_name").alias("stations"),
    avg("ride_duration_minutes").alias("avg_duration_min")
).orderBy(col("weekend_trips").desc()).limit(3)

weekend_geo.show(truncate=False)


7. География поездок в будни vs выходные:


 Топ-3 зон в будние дни:
+--------+--------+-------------+--------+------------------+
|lat_grid|lng_grid|weekday_trips|stations|avg_duration_min  |
+--------+--------+-------------+--------+------------------+
|41.88   |-87.64  |721657       |96      |13.58072556168189 |
|41.89   |-87.63  |624969       |96      |17.997765622508208|
|41.88   |-87.63  |546143       |125     |18.490661053973053|
+--------+--------+-------------+--------+------------------+


 Топ-3 зон в выходные дни:
+--------+--------+-------------+--------+------------------+
|lat_grid|lng_grid|weekend_trips|stations|avg_duration_min  |
+--------+--------+-------------+--------+------------------+
|41.89   |-87.63  |251031       |55      |28.689740443743332|
|41.88   |-87.62  |244017       |29      |47.56892040308666 |
|41.9    |-87.63  |232278       |61      |22.21350192441815 |
+--------+--------+-------------+--------+------------------+



In [47]:
# 10.8. Самые длинные маршруты по географическому расстоянию
print("\n8. Самые длинные маршруты по географическому расстоянию:")
print("-" * 40)

print("\n• Топ-10 маршрутов с наибольшим изменением координат:")

longest_distance_routes = df_with_geo_features.filter(
    col("coord_diff").isNotNull() &
    col("start_station_name").isNotNull() &
    col("end_station_name").isNotNull() &
    (col("start_station_name") != col("end_station_name"))
).groupBy("start_station_name", "end_station_name").agg(
    count("*").alias("route_count"),
    avg("coord_diff").alias("avg_coord_diff"),
    avg("ride_duration_minutes").alias("avg_duration_min"),
    max("coord_diff").alias("max_coord_diff")
).filter(col("route_count") >= 10).orderBy(col("avg_coord_diff").desc()).limit(10)

longest_distance_routes.show(truncate=False)


8. Самые длинные маршруты по географическому расстоянию:
----------------------------------------

• Топ-10 маршрутов с наибольшим изменением координат:
+-------------------------------+-------------------------------+-----------+-------------------+------------------+-------------------+
|start_station_name             |end_station_name               |route_count|avg_coord_diff     |avg_duration_min  |max_coord_diff     |
+-------------------------------+-------------------------------+-----------+-------------------+------------------+-------------------+
|Lakefront Trail & Bryn Mawr Ave|South Shore Dr & 71st St       |21         |0.30424142587069947|112.915873015873  |0.3043020014000035 |
|South Shore Dr & 71st St       |Lakefront Trail & Bryn Mawr Ave|25         |0.30422946443610926|122.794           |0.3042511650199984 |
|Shore Dr & 55th St             |Broadway & Thorndale Ave       |13         |0.27390406069794604|123.37051282051281|0.2739569999999958 |
|Lakefront Trail & Bryn 

In [48]:
# 11.1. Базовые метрики
print("\n1. ОСНОВНЫЕ МЕТРИКИ ПО СЕГМЕНТАМ:")
print()

user_comparison = df_with_features.groupBy("member_casual").agg(
    count("*").alias("total_rides"),
    (count("*") / row_count * 100).alias("percentage_of_total"),
    avg("ride_duration_minutes").alias("avg_duration_min"),
    stddev("ride_duration_minutes").alias("std_duration_min"),
    expr("percentile_approx(ride_duration_minutes, 0.5)").alias("median_duration_min"),
    expr("percentile_approx(ride_duration_minutes, 0.25)").alias("q1_duration_min"),
    expr("percentile_approx(ride_duration_minutes, 0.75)").alias("q3_duration_min"),
    countDistinct("rideable_type").alias("unique_bike_types"),
    countDistinct("start_station_name").alias("unique_start_stations"),
    countDistinct("end_station_name").alias("unique_end_stations")
).orderBy(col("total_rides").desc())

user_comparison.show(truncate=False)


1. ОСНОВНЫЕ МЕТРИКИ ПО СЕГМЕНТАМ:

+-------------+-----------+-------------------+------------------+------------------+-------------------+-----------------+---------------+-----------------+---------------------+-------------------+
|member_casual|total_rides|percentage_of_total|avg_duration_min  |std_duration_min  |median_duration_min|q1_duration_min  |q3_duration_min|unique_bike_types|unique_start_stations|unique_end_stations|
+-------------+-----------+-------------------+------------------+------------------+-------------------+-----------------+---------------+-----------------+---------------------+-------------------+
|member       |13046383   |42.06630823974048  |12.728148086459342|144.98547131339166|9.2                |5.3              |16.0           |3                |1954                 |1953               |
|casual       |8745312    |28.198082966344106 |31.796913592105113|301.5386409541647 |14.466666666666667 |7.983333333333333|27.4           |3                |2085   

In [52]:
# 11.2. Анализ по типам велосипедов
print("\n2. ПРЕДПОЧТЕНИЯ ПО ТИПАМ ВЕЛОСИПЕДОВ:")
print()

bike_type_analysis = df_with_features.groupBy("member_casual", "rideable_type").agg(
    count("*").alias("ride_count"),
    (count("*") / row_count * 100).alias("percentage_of_total"),
    avg("ride_duration_minutes").alias("avg_duration_min"),
    expr("percentile_approx(ride_duration_minutes, 0.5)").alias("median_duration_min")
).orderBy("member_casual", col("ride_count").desc())

print(" Распределение по типам велосипедов:")
bike_type_analysis.show(truncate=False)


2. ПРЕДПОЧТЕНИЯ ПО ТИПАМ ВЕЛОСИПЕДОВ:

 Распределение по типам велосипедов:
+-------------+-------------+----------+-------------------+------------------+-------------------+
|member_casual|rideable_type|ride_count|percentage_of_total|avg_duration_min  |median_duration_min|
+-------------+-------------+----------+-------------------+------------------+-------------------+
|casual       |electric_bike|3781003   |12.191335916888494 |16.28283805469961 |11.083333333333334 |
|casual       |classic_bike |3298656   |10.636072854282245 |30.468935171374728|15.2               |
|casual       |docked_bike  |1665653   |5.370674195173363  |69.64351797763415 |25.116666666666667 |
|member       |classic_bike |6180828   |19.929249035906633 |14.074859590009632|9.383333333333333  |
|member       |electric_bike|5423669   |17.48789162056066  |11.0178128833698  |8.366666666666667  |
|member       |docked_bike  |1441886   |4.649167583273191  |13.38874112331564 |12.383333333333333 |
+-------------+--------

In [53]:
# Процентное распределение внутри каждого сегмента
print("\n Процентное распределение внутри каждого сегмента:")

# Сначала посчитаем общее количество поездок по каждому сегменту
segment_totals = df_with_features.groupBy("member_casual") \
    .agg(count("*").alias("segment_total"))

# Посчитаем количество поездок по типам велосипедов
bike_counts = df_with_features.filter(col("rideable_type").isNotNull()) \
    .groupBy("member_casual", "rideable_type") \
    .agg(count("*").alias("type_count"))

# Объединяем и считаем проценты
bike_type_pct = bike_counts.join(segment_totals, "member_casual") \
    .withColumn("percentage_in_segment", 
               round(col("type_count") * 100.0 / col("segment_total"), 2)) \
    .select("member_casual", "rideable_type", "type_count", "percentage_in_segment") \
    .orderBy("member_casual", col("type_count").desc())

bike_type_pct.show(truncate=False)


 Процентное распределение внутри каждого сегмента:
+-------------+----------------+----------+---------------------+
|member_casual|rideable_type   |type_count|percentage_in_segment|
+-------------+----------------+----------+---------------------+
|casual       |electric_bike   |5841758   |47.68                |
|casual       |classic_bike    |4659611   |38.03                |
|casual       |docked_bike     |1665653   |13.59                |
|casual       |electric_scooter|85215     |0.7                  |
|member       |electric_bike   |8777995   |46.79                |
|member       |classic_bike    |8482613   |45.21                |
|member       |docked_bike     |1441886   |7.69                 |
|member       |electric_scooter|59122     |0.32                 |
+-------------+----------------+----------+---------------------+



In [54]:
# 11.3. Временные паттерны использования
print("\n3. ВРЕМЕННЫЕ ПАТТЕРНЫ ИСПОЛЬЗОВАНИЯ:")
print()

# По часам суток
print("\n Активность по часам суток (топ-5 часов для каждого сегмента):")

hourly_pattern = df_with_features.groupBy("member_casual", "ride_hour").agg(
    count("*").alias("ride_count"),
    avg("ride_duration_minutes").alias("avg_duration_min")
).orderBy("member_casual", col("ride_count").desc())

# Для каждого сегмента показываем топ-5 часов
for segment in ["member", "casual"]:
    segment_hours = hourly_pattern.filter(col("member_casual") == segment) \
                                  .limit(5)
    print(f"\n  Топ-5 часов для '{segment}' пользователей:")
    segment_hours.show(truncate=False)


3. ВРЕМЕННЫЕ ПАТТЕРНЫ ИСПОЛЬЗОВАНИЯ:


 Активность по часам суток (топ-5 часов для каждого сегмента):

  Топ-5 часов для 'member' пользователей:
+-------------+---------+----------+------------------+
|member_casual|ride_hour|ride_count|avg_duration_min  |
+-------------+---------+----------+------------------+
|member       |17       |1375135   |14.084633339029743|
|member       |16       |1149572   |13.814086546993124|
|member       |18       |1125981   |13.988422007120876|
|member       |15       |881807    |13.854346245833833|
|member       |19       |800298    |13.744080600809523|
+-------------+---------+----------+------------------+


  Топ-5 часов для 'casual' пользователей:
+-------------+---------+----------+------------------+
|member_casual|ride_hour|ride_count|avg_duration_min  |
+-------------+---------+----------+------------------+
|casual       |17       |837003    |29.42767349698867 |
|casual       |16       |750873    |31.35585540652903 |
|casual       |18       |7

In [55]:
# По дням недели
print("\n Активность по дням недели (1=Вс, 7=Сб):")

dow_analysis = df_with_features.groupBy("member_casual", "ride_dayofweek").agg(
    count("*").alias("ride_count"),
    avg("ride_duration_minutes").alias("avg_duration_min"),
    stddev("ride_duration_minutes").alias("std_duration_min")
).orderBy("member_casual", "ride_dayofweek")

print("Статистика по дням недели:")
dow_analysis.show(truncate=False)


 Активность по дням недели (1=Вс, 7=Сб):
Статистика по дням недели:
+-------------+--------------+----------+------------------+------------------+
|member_casual|ride_dayofweek|ride_count|avg_duration_min  |std_duration_min  |
+-------------+--------------+----------+------------------+------------------+
|casual       |1             |1531306   |37.49815760751488 |318.4298150791392 |
|casual       |2             |1000699   |31.603073451657306|271.9209285034107 |
|casual       |3             |976502    |25.817305119020112|355.101859610756  |
|casual       |4             |1013917   |27.64603249904415 |270.38617648247083|
|casual       |5             |1084836   |28.334954761211247|288.62612492308057|
|casual       |6             |1280775   |30.675068649840902|293.3581868211591 |
|casual       |7             |1857277   |35.40642554485244 |300.6799653133051 |
|member       |1             |1528723   |15.097524753230864|71.76047583985206 |
|member       |2             |1785680   |12.8325945

In [57]:
# Сравнение будни/выходные
print("\n Сравнение будни vs выходные:")

weekend_analysis = df_with_geo_features.groupBy("member_casual", "is_weekend").agg(
    count("*").alias("ride_count"),
    (count("*") / row_count * 100).alias("percentage_of_total"),
    avg("ride_duration_minutes").alias("avg_duration_min")
).orderBy("member_casual", "is_weekend")

print("Активность в будни (False) и выходные (True):")
weekend_analysis.show(truncate=False)


 Сравнение будни vs выходные:
Активность в будни (False) и выходные (True):
+-------------+----------+----------+-------------------+------------------+
|member_casual|is_weekend|ride_count|percentage_of_total|avg_duration_min  |
+-------------+----------+----------+-------------------+------------------+
|casual       |false     |5356729   |17.27205258888665  |28.91563796675161 |
|casual       |true      |3388583   |10.926030377457455 |36.35168256170782 |
|member       |false     |9741280   |31.409447900588166 |11.942904409550504|
|member       |true      |3305103   |10.656860339152313 |15.042532396116025|
+-------------+----------+----------+-------------------+------------------+



In [58]:
# 11.4. Сезонность и месячные паттерны
print("\n4. СЕЗОННОСТЬ И МЕСЯЧНЫЕ ПАТТЕРНЫ:")
print()

monthly_pattern = df_with_features.groupBy("member_casual", "ride_month").agg(
    count("*").alias("ride_count"),
    avg("ride_duration_minutes").alias("avg_duration_min")
).orderBy("member_casual", "ride_month")

print(" Активность по месяцам:")
monthly_pattern.show(30, truncate=False)


4. СЕЗОННОСТЬ И МЕСЯЧНЫЕ ПАТТЕРНЫ:

 Активность по месяцам:
+-------------+----------+----------+-------------------+
|member_casual|ride_month|ride_count|avg_duration_min   |
+-------------+----------+----------+-------------------+
|casual       |1         |101105    |24.388242586090414 |
|casual       |2         |121726    |26.763942515704716 |
|casual       |3         |318666    |29.90796267356204  |
|casual       |4         |565741    |32.09405629077613  |
|casual       |5         |1089439   |33.0561639675708   |
|casual       |6         |1195680   |35.50174595209412  |
|casual       |7         |1448765   |36.724038899798835 |
|casual       |8         |1372386   |33.76105408148044  |
|casual       |9         |1152914   |29.322912015408757 |
|casual       |10        |788314    |27.03525217531764  |
|casual       |11        |394192    |23.65810886573027  |
|casual       |12        |196384    |11.297824245695631 |
|member       |1         |434673    |12.08332413101345  |
|member    

In [59]:
# 11.5. Географические различия
print("\n5. ГЕОГРАФИЧЕСКИЕ РАЗЛИЧИЯ:")
print()

# Популярные станции для каждого сегмента
print("\n Топ-5 станций отправления для каждого сегмента:")

for segment in ["member", "casual"]:
    top_stations = df_with_features.filter(col("member_casual") == segment) \
        .filter(col("start_station_name").isNotNull()) \
        .groupBy("start_station_name") \
        .agg(
            count("*").alias("departure_count"),
            avg("ride_duration_minutes").alias("avg_duration_min")
        ) \
        .orderBy(col("departure_count").desc()) \
        .limit(5)
    
    print(f"\n  Топ-5 станций для '{segment}' пользователей:")
    top_stations.show(truncate=False)


5. ГЕОГРАФИЧЕСКИЕ РАЗЛИЧИЯ:


 Топ-5 станций отправления для каждого сегмента:

  Топ-5 станций для 'member' пользователей:
+----------------------------+---------------+------------------+
|start_station_name          |departure_count|avg_duration_min  |
+----------------------------+---------------+------------------+
|Clark St & Elm St           |97485          |12.1568566104187  |
|Kingsbury St & Kinzie St    |96093          |8.853021898924306 |
|Wells St & Concord Ln       |86092          |11.93421204447955 |
|Wells St & Elm St           |79246          |11.353714803691453|
|Clinton St & Washington Blvd|77575          |11.556316897625958|
+----------------------------+---------------+------------------+


  Топ-5 станций для 'casual' пользователей:
+---------------------------------+---------------+------------------+
|start_station_name               |departure_count|avg_duration_min  |
+---------------------------------+---------------+------------------+
|Streeter Dr & Grand A

In [60]:
# 11.6. Анализ длительности поездок
print("\n6. ДЕТАЛЬНЫЙ АНАЛИЗ ДЛИТЕЛЬНОСТИ ПОЕЗДОК:")
print()

# Распределение поездок по категориям длительности
print("\n Категоризация поездок по длительности:")

duration_categories = df_with_features.withColumn(
    "duration_category",
    when(col("ride_duration_minutes") < 5, "0-5 мин")
    .when(col("ride_duration_minutes") < 15, "5-15 мин")
    .when(col("ride_duration_minutes") < 30, "15-30 мин")
    .when(col("ride_duration_minutes") < 60, "30-60 мин")
    .when(col("ride_duration_minutes") < 120, "1-2 часа")
    .otherwise(">2 часов")
).groupBy("member_casual", "duration_category").agg(
    count("*").alias("ride_count"),
    (count("*") / row_count * 100).alias("percentage_of_total")
).orderBy("member_casual", col("ride_count").desc())

duration_categories.show(truncate=False)


6. ДЕТАЛЬНЫЙ АНАЛИЗ ДЛИТЕЛЬНОСТИ ПОЕЗДОК:


 Категоризация поездок по длительности:
+-------------+-----------------+----------+-------------------+
|member_casual|duration_category|ride_count|percentage_of_total|
+-------------+-----------------+----------+-------------------+
|casual       |5-15 мин         |3499354   |11.283196576703965 |
|casual       |15-30 мин        |2306900   |7.438288947845338  |
|casual       |30-60 мин        |1207075   |3.892051077948941  |
|casual       |0-5 мин          |1008821   |3.252807704995571  |
|casual       |1-2 часа         |525788    |1.695332727604016  |
|casual       |>2 часов         |197374    |0.6364059312462724 |
|member       |5-15 мин         |6485301   |20.910981296003435 |
|member       |0-5 мин          |2963347   |9.554914057276276  |
|member       |15-30 мин        |2678154   |8.635347565489525  |
|member       |30-60 мин        |818370    |2.63872405663366   |
|member       |1-2 часа         |71038     |0.22905248180546933|
|memb

In [61]:
# 11.7. Анализ частоты использования
print("\n7. АНАЛИЗ ЧАСТОТЫ ИСПОЛЬЗОВАНИЯ (по уникальным пользователям):")
print()

print("\n Среднее количество поездок на одного пользователя (оценка):")

usage_intensity = df_with_features.groupBy("member_casual").agg(
    count("*").alias("total_rides"),
    countDistinct("ride_id").alias("unique_rides"),
    # Предполагаем, что каждый ride_id = одна поездка одного пользователя
    (count("*") / countDistinct("ride_id")).alias("avg_rides_per_user_est")
).orderBy(col("total_rides").desc())

usage_intensity.show(truncate=False)


7. АНАЛИЗ ЧАСТОТЫ ИСПОЛЬЗОВАНИЯ (по уникальным пользователям):


 Среднее количество поездок на одного пользователя (оценка):
+-------------+-----------+------------+----------------------+
|member_casual|total_rides|unique_rides|avg_rides_per_user_est|
+-------------+-----------+------------+----------------------+
|member       |18761616   |18761353    |1.000014018178753     |
|casual       |12252237   |12252080    |1.00001281415074      |
+-------------+-----------+------------+----------------------+



In [62]:
# 11.8. Анализ возвратов (поездки в ту же станцию)
print("\n8. АНАЛИЗ ВОЗВРАТОВ ВЕЛОСИПЕДОВ:")
print()

return_analysis = df_with_features.withColumn(
    "is_return_trip",
    when(col("start_station_name") == col("end_station_name"), 1).otherwise(0)
).groupBy("member_casual").agg(
    count("*").alias("total_rides"),
    sum("is_return_trip").alias("return_trips"),
    (sum("is_return_trip") / count("*") * 100).alias("return_rate_percent"),
    avg(when(col("is_return_trip") == 1, col("ride_duration_minutes"))).alias("avg_return_duration"),
    avg(when(col("is_return_trip") == 0, col("ride_duration_minutes"))).alias("avg_oneway_duration")
).orderBy("member_casual")

print(" Статистика по возвратам велосипедов:")
return_analysis.show(truncate=False)


8. АНАЛИЗ ВОЗВРАТОВ ВЕЛОСИПЕДОВ:

 Статистика по возвратам велосипедов:
+-------------+-----------+------------+-------------------+-------------------+-------------------+
|member_casual|total_rides|return_trips|return_rate_percent|avg_return_duration|avg_oneway_duration|
+-------------+-----------+------------+-------------------+-------------------+-------------------+
|casual       |8745312    |826758      |9.453727894442189  |47.918692773459725 |30.113675729179807 |
|member       |13046383   |495361      |3.7969221047703416 |14.419797548319938 |12.661382434301633 |
+-------------+-----------+------------+-------------------+-------------------+-------------------+



In [63]:
# 11.9. Анализ для монетизации
print("\n9. АНАЛИЗ ДЛЯ МОНЕТИЗАЦИИ И ЦЕНООБРАЗОВАНИЯ:")
print()

print("\n Оценка использования для ценообразования:")

# Процент поездок, которые превышают разные лимиты
for segment in ["member", "casual"]:
    segment_df = df_with_features.filter(col("member_casual") == segment)
    segment_count = segment_df.count()
    
    # Процент поездок разной длительности
    over_30_min = segment_df.filter(col("ride_duration_minutes") > 30).count()
    over_45_min = segment_df.filter(col("ride_duration_minutes") > 45).count()
    over_60_min = segment_df.filter(col("ride_duration_minutes") > 60).count()
    
    print(f"\n  Для '{segment}' пользователей:")
    print(f"   Поездок > 30 мин: {over_30_min:,} ({over_30_min/segment_count*100:.1f}%)")
    print(f"   Поездок > 45 мин: {over_45_min:,} ({over_45_min/segment_count*100:.1f}%)")
    print(f"   Поездок > 60 мин: {over_60_min:,} ({over_60_min/segment_count*100:.1f}%)")


9. АНАЛИЗ ДЛЯ МОНЕТИЗАЦИИ И ЦЕНООБРАЗОВАНИЯ:


 Оценка использования для ценообразования:

  Для 'member' пользователей:
   Поездок > 30 мин: 918,234 (4.9%)
   Поездок > 45 мин: 230,292 (1.2%)
   Поездок > 60 мин: 101,151 (0.5%)

  Для 'casual' пользователей:
   Поездок > 30 мин: 1,928,692 (15.7%)
   Поездок > 45 мин: 1,110,296 (9.1%)
   Поездок > 60 мин: 722,843 (5.9%)


In [64]:
# 13. ИТОГОВАЯ СВОДКА
print("\n")
print("ИТОГОВАЯ СВОДКА РАЗВЕДОЧНОГО АНАЛИЗА")
print()

summary = f"""
ОСНОВНЫЕ ВЫВОДЫ:
1. Объем данных: {row_count:,} поездок, {col_count} признаков
2. Временной период: с {time_range['first_ride']} по {time_range['last_ride']}
3. Типы пользователей: {', '.join([row[0] for row in df.select('member_casual').distinct().collect()])}
4. Типы велосипедов: {', '.join([row[0] for row in df.select('rideable_type').distinct().collect()])}
5. Средняя длительность поездки: {duration_stats['avg_duration_min']:.1f} минут
6. Пропущенные значения: присутствуют в {len([v for v in null_data.values() if v > 0])} из {col_count} столбцов
"""

print(summary)



ИТОГОВАЯ СВОДКА РАЗВЕДОЧНОГО АНАЛИЗА


ОСНОВНЫЕ ВЫВОДЫ:
1. Объем данных: 31,013,853 поездок, 13 признаков
2. Временной период: с 2020-04-01 00:00:30 по 2024-05-31 23:59:57
3. Типы пользователей: casual, member
4. Типы велосипедов: docked_bike, electric_bike, classic_bike, electric_scooter
5. Средняя длительность поездки: 20.4 минут
6. Пропущенные значения: присутствуют в 6 из 13 столбцов

